# Download responses and save them to SQLite

In [1]:
!pip install --upgrade citric pip pandas faker sqlalchemy --quiet

In [2]:
import io
from pathlib import Path

import pandas as pd
from faker import Faker
from sqlalchemy import create_engine

import citric

## Create citric client

In [3]:
# Use your own server's parameters here
client = citric.Client(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
)

## Load survey file

In [4]:
with Path("../../examples/survey.lss").open("rb") as f:
    survey_id = client.import_survey(f)

## Activate survey and load fake responses

In [5]:
faker = Faker()

client.activate_survey(
    survey_id,
    user_activation_settings={
        "datestamp": True,
    },
)
client.activate_tokens(survey_id)

data = [
    {
        "G01Q01": faker.text(max_nb_chars=100),
        "G01Q02": faker.random_int(1, 5),
        "token": faker.sha1()[:5],
        "ipaddr": faker.ipv4(),
    }
    for _ in range(100)
]

result = client.add_responses(survey_id, data)

In [6]:
with io.BytesIO() as file:
    file.write(client.export_responses(survey_id, file_format="csv"))
    file.seek(0)
    responses_df = pd.read_csv(
        file,
        delimiter=";",
        parse_dates=["datestamp", "startdate", "submitdate"],
        index_col="id",
    )

engine = create_engine("sqlite:///responses.db")

In [7]:
responses_df.head()

,submitdate,lastpage,startlanguage,seed,token,startdate,datestamp,G01Q01,G01Q02,G02Q03,G02Q03[filecount]
id,,,,,,,,,,,
1,2024-02-08 17:19:28,NaN,en,NaN,53afb,2024-02-08 17:19:28,2024-02-08 17:19:28,Score soldier network station edge. Degree mil...,5,NaN,NaN
2,2024-02-08 17:19:28,NaN,en,NaN,c36b3,2024-02-08 17:19:28,2024-02-08 17:19:28,Half college hospital. Sell matter two phone r...,3,NaN,NaN
3,2024-02-08 17:19:28,NaN,en,NaN,f99a1,2024-02-08 17:19:28,2024-02-08 17:19:28,Sense executive eye five fill. Technology hear...,2,NaN,NaN
4,2024-02-08 17:19:28,NaN,en,NaN,f8311,2024-02-08 17:19:28,2024-02-08 17:19:28,Score people half. Only center team care radio...,2,NaN,NaN
5,2024-02-08 17:19:28,NaN,en,NaN,676a2,2024-02-08 17:19:28,2024-02-08 17:19:28,Church clear of. Wear too way I. Expert everyt...,5,NaN,NaN


In [8]:
responses_df.to_sql("responses", engine, if_exists="replace")

100

In [9]:
client.delete_survey(survey_id)

{'status': 'OK'}